<a href="https://colab.research.google.com/github/Dvagm/DataScincePortfol.io/blob/main/ForecastingStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install bayesian-optimization


# Dataset
Siurce: yfinance is a Python library that allows users to access financial data from Yahoo Finance. It provides an easy-to-use interface for retrieving stock prices, historical market data, financial statements, and company information.



```
# import yfinance as yf
```



In [47]:
#Library
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from bayes_opt import BayesianOptimization
from keras.callbacks import Callback
from tqdm import tqdm
from tensorflow.keras.layers import Input


import torch
from transformers import TimeSeriesTransformerForPrediction, TimeSeriesTransformerConfig, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader


#Dataframe
A DataFrame is a two-dimensional, labeled data structure in pandas, similar to a table in a database, an Excel spreadsheet, or an R data frame.

In [22]:
# Ambil data saham Netflix (NFLX) dari Yahoo Finance
ticker = "NFLX"
df = yf.download(ticker, start="2005-01-01", end="2025-01-01")

# Preprocessing data
scaler = MinMaxScaler()
df['Close_Scaled'] = scaler.fit_transform(df[['Close']])

# Buat data untuk model
def create_sequences(data, seq_length=50):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 50
X, y = create_sequences(df['Close_Scaled'].values, seq_length)

[*********************100%***********************]  1 of 1 completed


In [45]:
df

Price,Close,High,Low,Open,Volume,Close_Scaled
Ticker,NFLX,NFLX,NFLX,NFLX,NFLX,
Date,,,,,,
2005-01-03,1.702857,1.800000,1.645714,1.800000,11334400,0.000441
2005-01-04,1.665714,1.707143,1.607143,1.702857,17352300,0.000402
2005-01-05,1.600000,1.677143,1.584286,1.677143,12732300,0.000331
2005-01-06,1.578571,1.624286,1.572857,1.600000,8273300,0.000309
2005-01-07,1.588571,1.650000,1.571429,1.587143,7490700,0.000319
...,...,...,...,...,...,...
2024-12-24,932.119995,935.849976,911.700012,915.000000,2320300,0.995253
2024-12-26,924.140015,930.489990,915.299988,928.400024,2340300,0.986720


# Data Preprocessing
Data preprocessing is the process of cleaning, transforming, and organizing raw data into a format that is suitable for analysis and machine learning models. It helps improve the accuracy, efficiency, and reliability of a model.

In [23]:
# Split data (80% training, 20% testing)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Tambahkan dimensi untuk LSTM
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [24]:
# Callback custom untuk progress bar
class TQDMProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.progress_bar = tqdm(total=self.epochs, desc="Training Progress", unit="epoch")

    def on_epoch_end(self, epoch, logs=None):
        self.progress_bar.update(1)
        self.progress_bar.set_postfix(loss=logs.get('loss', 0))

    def on_train_end(self, logs=None):
        self.progress_bar.close()


#Hypertunning
Hyperparameter tuning (or hyperparameter optimization) is the process of finding the best set of hyperparameters to improve a machine learning model's performance. Unlike model parameters (learned from data), hyperparameters are manually set before training.



In [28]:
# Fungsi untuk Bayesian Optimization
def train_lstm(batch_size, dropout_rate, epochs, lstm_units):
    batch_size = int(batch_size)
    epochs = int(epochs)
    lstm_units = int(lstm_units)

    model = Sequential([
    Input(shape=(seq_length, 1)),  # 📌 Gunakan Input layer di awal
    LSTM(lstm_units, return_sequences=True),
    Dropout(dropout_rate),
    LSTM(lstm_units),
    Dropout(dropout_rate),
    Dense(1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')

    # Latih model dengan tqdm progress bar
    progress_bar = TQDMProgressBar()
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[progress_bar])

    loss = model.evaluate(X_test, y_test, verbose=0)
    return -loss  # Bayesian Optimization memaksimalkan fungsi, jadi kita negasikan loss

# Hyperparameter tuning dengan Bayesian Optimization
pbounds = {
    'batch_size': (16, 64),
    'dropout_rate': (0.1, 0.5),
    'epochs': (10, 50),
    'lstm_units': (10, 100)
}

optimizer = BayesianOptimization(f=train_lstm, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=10)


|   iter    |  target   | batch_... | dropou... |  epochs   | lstm_u... |
-------------------------------------------------------------------------



Training Progress: 100%|██████████| 39/39 [03:11<00:00,  4.90s/epoch, loss=0.000502]


| 1         | -0.000942 | 33.98     | 0.4803    | 39.28     | 63.88     |



Training Progress:  67%|██████▋   | 26/39 [09:13<04:36, 21.30s/epoch, loss=0.000548]

Training Progress: 100%|██████████| 12/12 [01:51<00:00,  9.31s/epoch, loss=0.000192]


| 2         | -0.000479 | 23.49     | 0.1624    | 12.32     | 87.96     |


...

| 13        | -0.000757 | 25.29     | 0.3485    | 19.93     | 79.1      |


Training Progress: 100%|██████████| 49/49 [07:00<00:00,  8.58s/epoch, loss=0.000271]


| 14        | -0.001104 | 16.3      | 0.2837    | 49.07     | 57.41     |


Training Progress: 100%|██████████| 22/22 [00:49<00:00,  2.23s/epoch, loss=0.0011]


| 15        | -0.004254 | 51.64     | 0.31      | 22.01     | 11.77     |


In [29]:
# Dapatkan hyperparameter terbaik
best_params = optimizer.max['params']
best_params['epochs'] = int(best_params['epochs'])
best_params['batch_size'] = int(best_params['batch_size'])
best_params['lstm_units'] = int(best_params['lstm_units'])


#Use model

In [30]:
# Bangun model akhir dengan hyperparameter terbaik
model = Sequential([
    LSTM(best_params['lstm_units'], return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(best_params['dropout_rate']),
    LSTM(best_params['lstm_units']),
    Dropout(best_params['dropout_rate']),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:

# Latih model akhir
progress_bar = TQDMProgressBar()
history = model.fit(
    X_train, y_train,
    epochs=best_params['epochs'],
    batch_size=best_params['batch_size'],
    validation_data=(X_test, y_test),
    callbacks=[progress_bar],
    verbose=0
)

# Prediksi
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Training Progress: 100%|██████████| 28/28 [03:57<00:00,  8.48s/epoch, loss=0.00017]


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [39]:
# Inisialisasi array input dari data terakhir yang tersedia
last_sequence = df['Close_Scaled'].values[-seq_length:].reshape(1, seq_length, 1)

# Prediksi untuk 7 hari ke depan
future_predictions = []

for _ in range(7):
    next_prediction = model.predict(last_sequence, verbose=0)[0, 0]  # Ambil nilai prediksi
    future_predictions.append(next_prediction)

    # Perbarui urutan input dengan menambahkan prediksi baru dan menggeser data lama
    last_sequence = np.concatenate((last_sequence[:, 1:, :], np.array([[[next_prediction]]])), axis=1)

# Konversi hasil prediksi ke skala asli
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Buat DataFrame untuk hasil prediksi
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=7)
df_future = pd.DataFrame({'Date': future_dates, 'Predicted_Close': future_predictions.flatten()})

df_future

,Date,Predicted_Close
0,2025-01-01,860.925110
1,2025-01-02,843.610596
2,2025-01-03,826.178589
3,2025-01-04,810.336182
4,2025-01-05,796.424316
5,2025-01-06,784.210144
6,2025-01-07,773.303772


#Matrix evaluation

In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Ambil data aktual untuk 7 hari ke depan jika tersedia
actual_future_values = df['Close'].iloc[-7:].values

# Hitung metrik evaluasi
mae = mean_absolute_error(actual_future_values, future_predictions)
mse = mean_squared_error(actual_future_values, future_predictions)
rmse = np.sqrt(mean_squared_error(actual_future_values, future_predictions))
mape = np.mean(np.abs((actual_future_values - future_predictions.flatten()) / actual_future_values)) * 100

# Tampilkan hasil evaluasi
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


Mean Absolute Error (MAE): 97.30
Mean Squared Error (MSE): 10125.27
Root Mean Squared Error (RMSE): 100.62
Mean Absolute Percentage Error (MAPE): 10.66%
